# Homework 2
Ponwalai Chalermwattanatrai 65340500042

## Part 1: Setting up Cart-Pole Agent.


#### 1.RL Base class

**discretize_state**
- This function takes a continuous-valued state (obs) and maps it to a discrete state.
- The discretization is done based on discretize_state_weight, which defines the number of discrete bins for each state variable.
- If a value is out of bounds, it is mapped to the nearest boundary.

state bound
- cart position: [-3, 3]
- pole position: [-24 deg, 24 deg]
- cart velocity: [-5, 5]
- pole velocity: [-5, 5]

In [ ]:
def discretize_state(self, obs: dict):
    """
    Discretize the observation state.

    Args:
        obs (dict): Observation dictionary containing policy states.

    Returns:
        Tuple[pose_cart:int, pose_pole:int, vel_cart:int, vel_pole:int]: Discretized state.
    """

    # ========= put your code here =========#
    policy_tensor = obs['policy']  # {'policy': tensor([[-0.2300,  0.0700, -0.1187, -0.1686]], device='cuda:0')}

    # Extracting the four values
    value = [0,0,0,0]
    value[0] = policy_tensor[0, 0].item()  # First value
    value[1] = policy_tensor[0, 1].item()  # Second value
    value[2] = policy_tensor[0, 2].item()   # Third value
    value[3] = policy_tensor[0, 3].item()   # Fourth value
    discrete_value = [0,0,0,0]

    bound = [[-3, 3],
             [-np.deg2rad(24), np.deg2rad(24)],
             [-5, 5],
             [-5, 5]]

    for i in range(0,4):
        value[i] = max(bound[i][0], min(value[i], bound[i][1]))
        # Compute discrete bin
        discrete_value[i] = round((value[i] - bound[i][0]) * (self.discretize_state_weight[i] - 1) / (bound[i][1] - bound[i][0])) + 1

    return (discrete_value[0],
            discrete_value[1],
            discrete_value[2],
            discrete_value[3])
    # ======================================#

**get_discretize_action**
- This function selects an action based on an epsilon-greedy policy, if unifrom random < epsilon random action else pick action from action that give max expected q value in that state.



In [ ]:
def get_discretize_action(self, obs_dis) -> int:
        """
        Select an action using an epsilon-greedy policy.

        Args:
            obs_dis (tuple): Discretized observation.

        Returns:
            int: Chosen discrete action index.
        """
        # ========= put your code here =========#
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.num_of_action)
        elif self.control_type == ControlType.DOUBLE_Q_LEARNING:
            return int(np.argmax(self.qa_values[obs_dis]+self.qb_values[obs_dis]))
        return int(np.argmax(self.q_values[obs_dis]))
        # ======================================#

**mapping_action**
- map action from range [0, num_of_action] to action_range

In [ ]:
def mapping_action(self, action):
        """
        Maps a discrete action in range [0, n] to a continuous value in [action_min, action_max].

        Args:
            action (int): Discrete action in range [0, n]
            n (int): Number of discrete actions
        
        Returns:
            torch.Tensor: Scaled action tensor.
        """
        # ========= put your code here =========#
        min_action, max_action = self.action_range
        action_step = (max_action - min_action) / (self.num_of_action - 1)
        action_value = min_action + action * action_step

        return torch.tensor([[action_value]], dtype=torch.float32)
        # ======================================#

**decay_epsilon**
- decay epsilon value with exponential trend $epsilon = start epsilon * epsilon decay^{current episode}$

In [ ]:
def decay_epsilon(self):
        """
        Decay epsilon value to reduce exploration over time.
        """
        # ========= put your code here =========#
        self.epsilon = max(self.final_epsilon, self.epsilon * self.epsilon_decay)
        return self.epsilon
        # ======================================#

#### 2. Algorithm

**Monte Carlo**

This algorithm updates Q-values using all state-action pairs from an episode, storing data in the following variables:
- obs_hist : list of discretize state
- action_hist : list of action index
- reward_hist : list of reward value

Update equation is
- update cumulative reward: $G_t​=R_t​+{\gamma}G_{t+1​}$
    - $R_t$ is the reward at time step $t$.
    - ${\gamma}$ (discount factor) determines the important of future rewards.
    - $G$ is computed backward from the last step of the episode.

- update Q-values: $Q(s,a)=\frac{Q(s,a)⋅N(s,a)+G}{N(s,a)+1}$
    - $Q(s,a)$ is the Q-value for state-action pair $(s,a)$.
    - $N(s,a)$ is the visit count of $(s,a)$.

In [ ]:
def update(self):
    """
    Update Q-values using Monte Carlo.

    This method applies the Monte Carlo update rule to improve policy decisions by updating the Q-table.
    """
    G = 0
    for t in reversed(range(len(self.reward_hist))):
        state = self.obs_hist[t]
        action = int(self.action_hist[t])
        G = self.reward_hist[t] + self.discount_factor * G

        # Check if the state-action pair is first visit
        if state not in self.obs_hist[:t]:
            self.q_values[state][action] = ((self.q_values[state][action] * (self.n_values[state][action])) + G) / (self.n_values[state][action] + 1)
            self.n_values[state][action] += 1
    self.obs_hist = []
    self.action_hist = []
    self.reward_hist = []

**SARSA**

This algorithm updates Q-values based on the action chosen by the current policy.

Update equation is:
- update Q-values: $Q(s,a)=Q(s,a)+{\alpha}[R+{\gamma}Q(s',a')−Q(s,a)]$
    - $Q(s,a)$ is the Q-value for the current state-action pair.
    - ${\alpha}$ (learning rate) controls how much new information overrides old information.
    - $R$ is the reward received after taking action $a$.
    - ${\gamma}$ (discount factor) determines the importance of future rewards.
    - $Q(s',a')$ is the Q-value of the next state-action pair (following the policy).

In [ ]:
def update(self, state, action, reward, next_state, next_action):
    """
    Update Q-values using SARSA .

    This method applies the SARSA update rule to improve policy decisions by updating the Q-table.
    """
    q_value = self.q_values[state][action]
    next_q_value = self.q_values[next_state][next_action] if next_state is not None else 0
    self.q_values[state][action] = q_value + self.lr*(reward + (self.discount_factor * next_q_value) - q_value)
    self.n_values[state][action] += 1

**Q learning**

This algorithm updates Q-values based on the max Q-values in next state.

Update equation is:
- update Q-values: $Q(s,a)=Q(s,a)+{\alpha}[R+{\gamma}\underset{a'}{\max}Q(s',a')−Q(s,a)]$
    - $Q(s,a)$ is the Q-value for the current state-action pair.
    - ${\alpha}$ (learning rate) controls how much new information overrides old information.
    - $R$ is the reward received after taking action $a$.
    - ${\gamma}$ (discount factor) determines the importance of future rewards.
    - $\underset{a'}{\max}Q(s',a')$ is the maximum Q-value in the next state (off policy).

In [ ]:
def update(self, state, action, reward, next_state):
        """
        Update Q-values using Q-Learning.

        This method applies the Q-Learning update rule to improve policy decisions by updating the Q-table.
        """
        q_value = self.q_values[state][action]
        max_next_q_value = np.max(self.q_values[next_state]) if next_state is not None else 0
        self.q_values[state][action] = q_value + self.lr*(reward + (self.discount_factor * max_next_q_value) - q_value)
        self.n_values[state][action] += 1

**Double Q learning**

This algorithm mitigates the overestimation bias in Q-learning by maintaining two separate Q-value tables ($Q_a$ and $Q_b$).

Update equation is:
- update $Q_a$: $Q_a(s,a)=Q_a(s,a)+{\alpha}[R+{\gamma}\underset{a'}{\max}Q_b(s',a')−Q_a(s,a)]$
- update $Q_b$: $Q_b(s,a)=Q_b(s,a)+{\alpha}[R+{\gamma}\underset{a'}{\max}Q_a(s',a')−Q_b(s,a)]$
    - $Q_a(s,a), Q_b(s,a)$ is the Q-value for the current state-action pair from two separate tables.
    - ${\alpha}$ (learning rate) controls how much new information overrides old information.
    - $R$ is the reward received after taking action $a$.
    - ${\gamma}$ (discount factor) determines the importance of future rewards.
    - $\underset{a'}{\max}Q_a(s',a'), \underset{a'}{\max}Q_b(s',a')$ is the maximum Q-value in the next state from two separate table (off policy).
    - At each step, either $Q_a$ or $Q_b$ is updated, chosen randomly with a probability of 0.5.

In [ ]:
def update(self, state, action, reward, next_state):
        """
        Update Q-values using Double Q-Learning.

        This method applies the Double Q-Learning update rule to improve policy decisions by updating the Q-table.
        """
        if np.random.random() < 0.5:
            # Update Q_a
            max_next_q_value = np.max(self.qb_values[next_state]) if next_state is not None else 0
            q_value = self.qa_values[state][action]
            self.qa_values[state][action] = q_value + self.lr * (reward + (self.discount_factor * max_next_q_value) - q_value)
            self.na_values[state][action] += 1
        else:
            # Update Q_b
            max_next_q_value = np.max(self.qa_values[next_state]) if next_state is not None else 0
            q_value = self.qb_values[state][action]
            self.qb_values[state][action] = q_value + self.lr * (reward + (self.discount_factor * max_next_q_value) - q_value)
            self.nb_values[state][action] += 1

## Part 2: Trainning & Playing to stabilize Cart-Pole Agent.

result of playing cart-pole stabilize task

<video controls src="img/2025-03-26 01-06-49.mp4" title="Title"></video>

## Part 3: Evaluate Cart-Pole Agent performance.

The goal of this evaluation is to determine the learning efficiency of each reinforcement learning algorithm (Monte-Carlo, SARSA, Q-learning, Double Q-learning) under varying hyperparameter conditions on the given cart-pole task.

**Environment Configuration**
- Reward Terms

    | Index | Name        | Weight |
    |-------|------------|--------|
    | 0     | alive      | 1.0    |
    | 1     | terminating | -2.0   |
    | 2     | pole_pos   | -1.0   |

- Termination Terms

    | Index | Name               | Bound |
    |-------|--------------------|------------------|
    | 0     | time_out           |                  |
    | 1     | cart_out_of_bounds | [-3, 3]          |
    | 2     | pole_out_of_bounds | [-24, 24] deg    |

- Event Terms (reset cart-pole after terminated)

    | Index | Name                  | Bound |
    |-------|-----------------------|----------------|
    | 0     | reset_cart_position   |pos:[-1,1] vel:[-0.5,0.5]|
    | 1     | reset_pole_position   |pos:[-24,24] vel:[-24,24]|

- State Observation Terms

    | Index | Name           | Shape  |
    |-------|---------------|--------|
    | 0     | joint_pos_rel | (2,)   |
    | 1     | joint_vel_rel | (2,)   |

- Action Terms

    | Index | Name         | Dimension |
    |-------|-------------|-----------|
    | 0     | joint_effort | 1         |

**Experiment**

In this part is to analyze the agent's performance in terms of learning efficiency of each algorithm in different hyperparameters, So in this part will set to 3 experiment
- minimize shape of observation state: To find the best 2 state from 4 state for visualize.
- Adjust learning rate and discount factor: To see impact of learning rate and discount factor to each algorithm and find best learning rate and discount factor for this cart-pole task
- Adjust size of action space and observation space: To see how size of action space and observation space impact to each algorithm which algorithm perform good in which size and find which algorithm perform best in this cart-pole task

**Experiment Conditions**

Fixed Hyperparameters Across Experiments:
- action_range = [-15,15] (this one is perform best and fit to this task)
- n_episodes = 5000
- start_epsilon = 1.0
- epsilon_decay = 0.9995
- final_epsilon = 0.1

#### 3.1. minimize shape of observation state

This experiment aims to simplify the visualization for the next experiments by reducing the number of observation states. Specifically, we compare a 4-state observation with different 2-state observation combinations to determine which performs best.

**Perfroming results**
- policy visualization

<img src="img/image.png" width="1000px"/>
<img src="img/image5.png" width="1000px"/>

- reward visualization

<img src="img/image2.png" width="1000px"/>

From the graphs, we observe the following:
- The best performing combination which highest returns and clear policy trends is pole position (pole-pos) and pole velocity (pole-vel).
- However, when visualizing in cart-pole task performs, we notice that pole-pos with pole-vel focuses too much on the pole, causing oscillations and preventing a steady-state balance.
- On the other hand, pole-pos with cart velocity (vel-cart), which ranks second in reward and policy trends, performs better in the cart-pole task. This combination avoids excessive focus on the pole and achieves better overall balance.

**Conclusion**
For future experiments, we will use pole-pos with vel-cart as the observation state. This combination provides better performance in the cart-pole task while preventing excessive focus on the pole, leading to more stable control.

#### 3.2 Adjust learning rate and discount factor

This experiment is aims to evaluate how learning rate and discount factor affect each algorithm’s performance. And which learning rate and discount factor perform best in cart-pole task

Parameters to vary:
- Learning rates (${\alpha}$): [0.1, 0.9]
- Discount factors (${\gamma}$): [0.7, 0.99]

**Perfroming results**
- policy visualization

    <img src="img/image3.png" width="1000px"/>
    <img src="img/image6.png" width="1000px"/>

- reward visualization
    - Monte-Carlo

    <img src="img/imageMC.png" width="1000px"/>

    - SARSA

    <img src="img/imageSARSA.png" width="1000px"/>

    - Q-learning

    <img src="img/imageQ.png" width="1000px"/>

    - Double Q-learning

    <img src="img/imageDQ.png" width="1000px"/>

**Discount Factor**
- Observation:
    - Higher discount factor (${\gamma}$ = 0.99) tends to perform worse in reward, meaning the episode ends sooner -> the agent fails earlier.
    - max Q-value graph: At high discount, the policy tries to control both pole angle and cart velocity, aiming for a long-term stable balance. At low discount (${\gamma}$ = 0.7), the policy only focuses on pole angle, regardless of cart velocity -> indicates preference for immediate correction.

- Interpretation:
    - Higher ${\gamma}$ = more long-term reward -> more complex behavior, harder to learn without exploration.
    - Lower ${\gamma}$ = favors short-term reward -> quicker convergence and works better in this task where survival is short-term reactive (keep pole upright now).

**Learning Rate**
- Observation:
    - Higher learning rate (${\alpha}$ = 0.9) reaches better or similar reward in some config compare to lower ${\alpha}$. In complex task learning faster than lower ${\alpha}$.
    - max Q-value graph becomes flatter, showing faster convergence, but sometimes less precise trends.

- Interpretation:
    - Higher ${\alpha}$ = faster learning but each update changes the values a lot, so the learned policy might be less accurate or more random-looking.
    - Lower ${\alpha}$ = the agent updates more slowly, so the value it learns is more careful and detailed, leading to a smoother and more precise policy.

#### 3.3 Adjust size of action space and observation space

This experiment is aims to evaluate how resolution of action space and observation space affect each algorithm’s performance.

Parameters to vary:
- num_of_action (discretize action to x value): [3, 11]
- discretize_state_weight (discretize observation to x value): [[1,5,5,1], [1,21,21,1]] ([pose_cart:int, pose_pole:int, vel_cart:int, vel_pole:int])

**Perfroming results**
- policy visualization

    <img src="img/image4.png" width="1000px"/>
    <img src="img/image7.png" width="1000px"/>

- reward visualization
    - Monte-Carlo

    <img src="img/imageMC2.png" width="1000px"/>

    - SARSA

    <img src="img/imageSARSA2.png" width="1000px"/>

    - Q-learning

    <img src="img/imageQ2.png" width="1000px"/>

    - Double Q-learning

    <img src="img/imageDQ2.png" width="1000px"/>

- Cart-pole task Perform
    <video controls src="img/video2.mp4" title="Title"></video>

    - Cart-pole task Perform graph
    <img src="img/image9.png" width="1000px"/>
    <img src="img/image8.png" width="1000px"/>

From graph we will see that
| Action size | State size | Characteristic  |
|-------------|------------|-----------------|
| 3           | 5          |Coarsest setting; fastest to compute, lowest reward, more noise (except MC which is stable)|
| 11          | 5          |Better than 3x5; less noisy reward, slightly better learning|
| 3           | 21         |High state resolution improves reward obvously but takes more time|
| 11          | 21         |Highest resolution; more precise but harder to train (SARSA & Double Q struggle)|

**Action Resolution**
- Observation:
    - Higher action resolution (more discrete actions like size 11) gives slightly better rewards than lower resolution (size 3), across all algorithms.
    - With lower actions size, the Q-value trends become flat, especially in SARSA and Q-learning.

- Interpretation:
    - Higher action resolution allows the agent to choose actions that better match the environment’s needs, leading to more precise control and smoother policy.
    - Lower action resolution forces the agent to choose from fewer options, making control less precise and leading to more noisy learning.
    - In higher state resolution higher action will make agent more state-action pair make agent use more time. So changing action resolution is trade off between speed and precise.

**State Resolution**
- Observation:
    - Higher state resolution leads to higher rewards but it use more time to compute.
    - Higher state Resolution also leads to more complex Q-value graph.

- Interpretation:
    - High state resolution gives the agent more detailed information about the environment and it make agent can fine-tune actions more accurately and make high reward.
    - Lower state resolution simplifies the state to small size make agent learn quicklier but causes the agent to group many situations into one state, make agent cannot fine tune to steady state and receive less reward.
    - Higher state resolution make agent want more explore time to find all state and also use more overall time. So changing state resolution is also trade off between speed and precise.

#### **Compare each algorithm**
- Sampling vs Bootstrap (MC vs Q-learning)
    - MC is always leds **policy to update stable and smooth** in every situation because it update from real reward.
    - Q-learning is update evrey step make agent more aggressive leading to highest reward making **policy more curved trend direction** but also make policy not smooth.
- on-policy vs off-policy (SARSA vs Q-learning)
    - SARSA is updates Q-values step-by-step using the next action chosen from epsilon greedy, If that action is random (due to exploration), **SARSA may learn from a non-optimal** or misleading reward making non-optimal policy.
    - Q learning is have higher reward and also **can compute to more situation** because it calculate from actual exploit reward so it directly q-value to go to highest reward direction and with balance explore and exploit will make Q-learning better than SARSA.
    - SARSA is better than Q-learning at **avoiding low-reward areas** because it updates using both random and greedy actions. If a random move leads to a risky zone, SARSA lowers the value of that state, encouraging the agent to avoid it.
    - In contrast, Q-learning always uses the best next action, so nearby low-reward states might still appear valuable. This is reflected in the policy graphs, where SARSA shows lower Q-values near edges, learning to avoid danger more effectively.
- Q-learning vs Double Q-learning
    - Double Q-learning have less bias than Q-learning make agent **use more time** to update both table if it update just 1 table will make agent might more trust in value that update to 2 table because higher value.
    - In this task Q-learning is better because it balance explore and exploit and **task didn't complex** so bias is lead agent to correct direction.

| Algorithm        | Pros                                                                 | Cons                                                                      |
|------------------|----------------------------------------------------------------------|---------------------------------------------------------------------------|
| **Monte Carlo**  | - Stable and smooth policy updates compare to bootstrap              | - Slower learning than Q-learning and SARSA (updates only at episode end) |
|                  | - Accurate value estimates (uses real total reward)                  |                                                                           |
| **SARSA**        | - Learns to avoid low-reward or risky states (better than Q-learning)| - Slower to reach high reward than Q-learning                             |
|                  |                                                                      | - Can learn from suboptimal actions (random action from ε-greedy) and make policy non-optimal |
| **Q-learning**   | - Faster learning than SARSA and MC (updates every step with greedy next action)       | - Policy surface less smooth than MC                                   |
|                  | - High reward performance in simple environments (bias helps when task is not complex) | - May ignore nearby risky areas (focuses only on best-case outcome)    |
|                  | - More direct policy trend (clear direction in value graph)                            |                                                                        |
| **Double Q**     | - Lower bias than Q-learning                                                           | - Slower than Q-learning (requires more steps to update both Q-tables) |
|                  | - More stable in complex or noisy environments                                         | - Needs more exploration to fully learn both Q-tables                  |


In conclusion the best algorithm is MC because MC have best perform in cart-pole task from it stable update policy and smooth of policy that make every episode MC can make pole stable in equally time. And task is small task so it doesn't store lot data or use very long time to update Monte-Carlo compare to others.

For Q-learning that have better reward it also perform good in cart-pole task but there are some unstable in some episode which is impact from aggressive learning that make policy not smooth and it better than SARSA and Double Q-learning because task is have less complex so bias from Q-learning can make agent learn faster and have highest reward and performance.